In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
# Define data transformations
data_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
# Load SVHN dataset
full_train_data = datasets.SVHN(root='./data', split='train', transform=data_transform, download=True)

# Take 25% of the dataset
subset_indices = np.random.choice(len(full_train_data), size=int(0.25 * len(full_train_data)), replace=False)
train_data = Subset(full_train_data, subset_indices)

test_data = datasets.SVHN(root='./data', split='test', transform=data_transform, download=True)


100%|██████████| 182040794/182040794 [00:19<00:00, 9235790.52it/s] 


100%|██████████| 64275384/64275384 [00:08<00:00, 7291559.67it/s]


In [4]:
# Create data loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


In [5]:
# Define the models
models_to_try = {
    #"AlexNet": models.alexnet,
    "VGG": models.vgg16,
    "ResNet-18": models.resnet18,
    "ResNet-50": models.resnet50,
    "ResNet-101": models.resnet101
}

In [6]:
# Function to train the model
def train_model(model, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")


In [7]:
# Function to evaluate the model
def evaluate_model(model):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Accuracy: {accuracy:.4f}")


In [8]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [9]:
# Training loop
for model_name, model_fn in models_to_try.items():
    print(f"Training {model_name}...")
    model = model_fn(pretrained=True)
    if "vgg" in model_name:
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, 10)
    elif "resnet" in model_name:
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 10)  # Adjust final layer for 10 classes
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    train_model(model, criterion, optimizer)
    print(f"Evaluating {model_name}...")
    evaluate_model(model)
    print()

Training VGG...


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 69.5MB/s]


Epoch 1/5, Loss: 1.5471
Epoch 2/5, Loss: 0.5283
Epoch 3/5, Loss: 0.3679
Epoch 4/5, Loss: 0.2922
Epoch 5/5, Loss: 0.2318
Evaluating VGG...
Accuracy: 0.9117

Training ResNet-18...


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 81.1MB/s]


Epoch 1/5, Loss: 2.3172
Epoch 2/5, Loss: 0.8394
Epoch 3/5, Loss: 0.5587
Epoch 4/5, Loss: 0.4155
Epoch 5/5, Loss: 0.3341
Evaluating ResNet-18...
Accuracy: 0.8528

Training ResNet-50...


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 94.7MB/s]


Epoch 1/5, Loss: 1.9902
Epoch 2/5, Loss: 0.5836
Epoch 3/5, Loss: 0.3518
Epoch 4/5, Loss: 0.2310
Epoch 5/5, Loss: 0.1600
Evaluating ResNet-50...
Accuracy: 0.8842

Training ResNet-101...


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 97.8MB/s]


Epoch 1/5, Loss: 1.9403
Epoch 2/5, Loss: 0.5890
Epoch 3/5, Loss: 0.3460
Epoch 4/5, Loss: 0.2206
Epoch 5/5, Loss: 0.1631
Evaluating ResNet-101...
Accuracy: 0.8730

